In [1]:
import string
from string import digits
%matplotlib inline
import re
import numpy as np 
import pandas as pd
import csv
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense, RepeatVector
from keras.models import Model

In [2]:
data= pd.read_csv("Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [4]:
data['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [5]:
data=data[data['source']=='tides'] #limiting to one resource
data.head()

,source,english_sentence,hindi_sentence
5,tides,The then Governor of Kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
9,tides,You may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...
10,tides,Please ensure that you use the appropriate form .,कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प...
14,tides,The first two were found unreliable and the pr...,पहले दो को अविश्वसनीय मानकर बाकी पांच मुखबिरों...
15,tides,They had justified their educational policy of...,कम संख़्या वाले उच्च एवं मध्यम श्रेणी के लोगों...


In [6]:
data.drop('source',axis=1,inplace=True) 
data.head()

,english_sentence,hindi_sentence
5,The then Governor of Kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
9,You may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...
10,Please ensure that you use the appropriate form .,कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प...
14,The first two were found unreliable and the pr...,पहले दो को अविश्वसनीय मानकर बाकी पांच मुखबिरों...
15,They had justified their educational policy of...,कम संख़्या वाले उच्च एवं मध्यम श्रेणी के लोगों...


In [7]:
data= data.dropna(axis=1)# dropping null values

In [8]:
data.drop_duplicates(inplace=True) #dropping duplicates

In [9]:
data['english_sentence']= data['english_sentence'].str.lower() # converting to lower case for uniformity, no need to do this for Hindi
data.head()

,english_sentence,hindi_sentence
5,the then governor of kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
9,you may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...
10,please ensure that you use the appropriate form .,कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प...
14,the first two were found unreliable and the pr...,पहले दो को अविश्वसनीय मानकर बाकी पांच मुखबिरों...
15,they had justified their educational policy of...,कम संख़्या वाले उच्च एवं मध्यम श्रेणी के लोगों...


In [10]:
data['english_sentence']=data['english_sentence'].str.replace('[^A-Za-z\s]+','') #Regex expression

<ipython-input-10-87150b51f361>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['english_sentence']=data['english_sentence'].str.replace('[^A-Za-z\s]+','') #Regex expression


In [11]:
data['english_count']= data['english_sentence'].str.len() #words in a single sentence
data['hindi_count']= data['hindi_sentence'].str.len() 
data.head()

,english_sentence,hindi_sentence,english_count,hindi_count
5,the then governor of kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...,110,112
9,you may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...,159,192
10,please ensure that you use the appropriate form,कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प...,48,65
14,the first two were found unreliable and the pr...,पहले दो को अविश्वसनीय मानकर बाकी पांच मुखबिरों...,123,76
15,they had justified their educational policy of...,कम संख़्या वाले उच्च एवं मध्यम श्रेणी के लोगों...,210,179


In [12]:
data=data[data['english_count']<=100] # both english and hindi count should be less than 100 to maintain uniformity in data
data=data[data['hindi_count']<=100]
data.head()

,english_sentence,hindi_sentence,english_count,hindi_count
10,please ensure that you use the appropriate form,कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प...,48,65
20,of these lahadi is a popular one,' लाडड़ी लोकप्रिय स्त्री नृत्य है .,33,35
21,even a concentration of ppm of hydrogen sulph...,यहां तक कि पानी में हाइड्रोजन सल्फाइड की 0.001...,96,99
28,all those who stand today under this flag are ...,"जो भी इस झंडे के नीचे खड़ा है वह न हिंदू है , ...",92,83
36,head lice need to maintain contact with a host...,जिंदा रहने के लिए जूँ किसी जीव के सम्पर्क में ...,67,60


In [13]:
data.shape

(18675, 4)

In [14]:
max_len_seq_eng=max(data['english_count'])
max_len_seq_hindi= max(data['hindi_count'])
max_len_seq_hindi

100

In [15]:
#maintaining a list of all unique english words
english_words=[]
for sent in data['english_sentence']:
    for word in sent.split():
        if word not in english_words:
            english_words.append(word)

In [16]:
english_words[:25]

['please',
 'ensure',
 'that',
 'you',
 'use',
 'the',
 'appropriate',
 'form',
 'of',
 'these',
 'lahadi',
 'is',
 'a',
 'popular',
 'one',
 'even',
 'concentration',
 'ppm',
 'hydrogen',
 'sulphide',
 'in',
 'water',
 'can',
 'emit',
 'smell']

In [17]:
#maintaining a list of all unique hindi words
hindi_words=[]
for sent in data['hindi_sentence']:
    for word in sent.split():
        if word not in hindi_words:
            hindi_words.append(word)

In [18]:
hindi_words[:25]

['कृपया',
 'यह',
 'सुनिश्चित',
 'कर',
 'लें',
 'कि',
 'आप',
 'सही',
 'फॉर्म',
 'का',
 'प्रयोग',
 'रहें',
 'हैं',
 '.',
 "'",
 'लाडड़ी',
 'लोकप्रिय',
 'स्त्री',
 'नृत्य',
 'है',
 'यहां',
 'तक',
 'पानी',
 'में',
 'हाइड्रोजन']

In [19]:
n_u_eng_words= len(english_words)
n_u_hindi_words=len(hindi_words)
n_u_eng_words

18932

In [20]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [21]:
english_sent=[]
for sent in data['english_sentence']:
    english_sent.append(sent)
english_sent[:5]

['please ensure that you use the appropriate form ',
 'of these lahadi is a popular one ',
 'even a concentration of  ppm of hydrogen sulphide in the water can emit the smell of rotten egg ',
 'all those who stand today under this flag are indians  not hindus  not muslims  but indians ',
 'head lice need to maintain contact with a host in order to survive ']

In [22]:
hindi_sent=[]
for sent in data['hindi_sentence']:
    hindi_sent.append(sent)
hindi_sent[:5]

['कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प्रयोग कर रहें हैं .',
 "' लाडड़ी लोकप्रिय स्त्री नृत्य है .",
 'यहां तक कि पानी में हाइड्रोजन सल्फाइड की 0.001 पी पी एम मात्रा से भी सड़े हुए अंडे की बदबू आती है .',
 'जो भी इस झंडे के नीचे खड़ा है वह न हिंदू है , न मुसलमान , बल्कि वह हिंदुस्तानी है .',
 'जिंदा रहने के लिए जूँ किसी जीव के सम्पर्क में रहना आवश्यक है']

In [23]:
# making tokenizer objects
tokenizer1= Tokenizer(lower=True, split=" ")
tokenizer2= Tokenizer(lower=True, split=" ")
tokenizer1.fit_on_texts(english_sent)
tokenizer2.fit_on_texts(hindi_sent)

In [24]:
word_index_hindi= tokenizer2.word_index

In [25]:
#making sequences 
eng_vector=tokenizer1.texts_to_sequences(english_sent)
hindi_vector=tokenizer2.texts_to_sequences(hindi_sent)

In [26]:
#padding 
eng_vect_seq= pad_sequences(eng_vector,maxlen=100,padding='post')
hindi_vect_seq= pad_sequences(hindi_vector, maxlen=100, padding='post')

In [27]:
# define Training and testing data set
x= eng_vect_seq
y= hindi_vect_seq
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.01, random_state=42)

# Defining model 

from keras.models import Sequential 
model= Sequential()
model.add(Embedding(n_u_eng_words+1,110,input_length=100,mask_zero=True))
model.add(LSTM(110))
model.add(RepeatVector(100))
model.add(LSTM(110,return_sequences=True))
model.add(Dense(n_u_hindi_words,activation='softmax'))

rms= keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(optimizer=rms,loss='sparse_categorical_crossentropy')


model.summary()

x_train=np.array(x_train)
y_train=np.array(y_train)

Translation_model= model.fit(x_train,y_train.reshape(y_train.shape[0],y_train.shape[1],1),epochs=10,batch_size=110,validation_split=0.2)

model.save('NMTmodel')

In [28]:
from keras.models import load_model
mymodel=load_model('NMTmodel')

In [35]:
sent=['yes']
tokenizer1.fit_on_texts(sent)
sent_vect=tokenizer1.texts_to_sequences(sent)
sent_vect_seq= pad_sequences(sent_vect,maxlen=100,padding='post')

In [36]:
pred=mymodel.predict(sent_vect_seq.reshape(sent_vect_seq.shape[0],sent_vect_seq.shape[1]))

In [37]:
classes= np.argmax(pred,axis=1)

In [38]:
def get_word(n,di):
    for key,value in di.items():
        if value==n:
            return key
    return None
        

In [ ]:
pred_text=[]
for i in classes:
    temp=[]
    for j in range(len(i)):
        t=get_word(i[j],word_index_hindi)
        if j>0:
            if t==get_word(i[j-1],word_index_hindi) or t==None:
                temp.append('')
            else:
                temp.append(t)
        else:
            if t==None:
                temp.append('')
            else:
                temp.append(t)
    print(' '.join(temp))
    pred_text.append(' '.join(temp))